### Overview

This is the first step for data preparation.

Window images, as well as reindex them 

Input: original CT images

Output: Images with abdominal windowing


In [3]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk

import sys

sys.path.insert(0, '../../dataloaders/')
# import niftiio as nio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# set up directories for images
IMG_FOLDER="./amos22/imagesTr/"
SEG_FOLDER="./amos22/labelsTr/"
OUT_FOLDER="./data/temp/"

In [5]:
imgs = glob.glob(IMG_FOLDER + "/*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/*.nii.gz")) ]

pids = [pid.split("img0")[-1].split(".")[0] for pid in imgs]

In [6]:
# helper function
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [ ]:
scan_dir = OUT_FOLDER
LIR = -125
HIR = 275
os.makedirs(scan_dir, exist_ok = True)

reindex = 0
for img_fid, seg_fid, pid in zip(imgs, segs, pids):

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    array = sitk.GetArrayFromImage(img_obj)

    array[array > HIR] = HIR
    array[array < LIR] = LIR
    
    array = (array - array.min()) / (array.max() - array.min()) * 255.0
    
    # then normalize this
    
    wined_img = sitk.GetImageFromArray(array)
    wined_img = copy_spacing_ori(img_obj, wined_img)
    
    out_img_fid = os.path.join( scan_dir, f'image_{str(reindex)}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{str(reindex)}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(wined_img, out_img_fid, True) 
    sitk.WriteImage(seg_obj, out_lb_fid, True) 
    print("{} has been saved".format(out_img_fid))
    print("{} has been saved".format(out_lb_fid))
    reindex += 1
